## Тестовое задание

При подведении итогов опроса по работе аналитиков данных, который ранее запускали на телеграм-канале было решено использовать в качестве индикатора зарплат в каждой группе использовать некоторый вариант усечённого среднего: сначала все анкетируемые в каждой группе обрезаются по 0.2 и 0.8 квантилям, а затем находится среднее арифметическое.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Импортируем данные. Проверим их качество.

In [2]:
ds_data = pd.read_excel('C:/Users/Alex/Downloads/test.xlsx')

In [3]:
ds_data.head()

,id_ds,city,salary
0,1,St.Petersburg,93
1,2,RU,55
2,3,Moscow,361
3,4,Moscow,161
4,5,St.Petersburg,217


In [4]:
ds_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id_ds   1197 non-null   int64 
 1   city    1197 non-null   object
 2   salary  1197 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 28.2+ KB


In [5]:
ds_data.duplicated().sum()

0

В данных нет пропусков и явных дубликтов.

In [6]:
ds_data['city'].unique()

array(['St.Petersburg', 'RU', 'Moscow', 'West', 'exCCCP'], dtype=object)

Проанализируем зарплаты для разных регионов.

In [7]:
ds_data.pivot_table(values='salary',
                   index='city',
                   aggfunc='describe')

,25%,50%,75%,count,max,mean,min,std
city,,,,,,,,
Moscow,90.25,185.0,323.50,790.0,553.0,224.915190,30.0,155.770448
RU,34.00,98.0,267.00,117.0,399.0,145.863248,10.0,132.360395
St.Petersburg,36.00,99.0,195.50,155.0,326.0,122.922581,10.0,98.173352
West,32.00,366.0,783.25,60.0,991.0,401.283333,10.0,361.723140
exCCCP,37.00,69.0,225.50,75.0,388.0,135.666667,11.0,121.167556


Из таблицы видно, что заплаты в Москве значительно больше, чем в других городах России и СНГ. Особенно в первом квантиле зарплаты самые высокие в выборке.    
Расчитаем предложенный вариант усеченного среднего. Не применяя циклы!

In [8]:
ds_data.set_index('id_ds',inplace=True,drop=False)

In [9]:
def quantile(row):
    bin_labels = ['1', '2', '3','4','5' ]
    city = row.loc['city']
    id_ds = row.loc['id_ds']
    quantile_number = pd.qcut(ds_data.query('city == @city')['salary'], q=5, labels=bin_labels)
    row['quantile_number'] = quantile_number.loc[id_ds]
    
    return row

In [10]:
ds_data = ds_data.apply(quantile,axis=1)

In [11]:
ds_data.head()

,id_ds,city,salary,quantile_number
id_ds,,,,
1,1,St.Petersburg,93,3
2,2,RU,55,3
3,3,Moscow,361,4
4,4,Moscow,161,3
5,5,St.Petersburg,217,4


In [12]:
cut_ds_data = ds_data.query('quantile_number not in ["1","5"]').dropna()

In [13]:
cut_ds_data.groupby('quantile_number')['salary'].describe()

,count,mean,std,min,25%,50%,75%,max
quantile_number,,,,,,,,
2,237.0,89.345992,41.990497,27.0,45.0,91.0,123.0,158.0
3,239.0,166.251046,68.049875,49.0,122.5,177.0,195.0,513.0
4,238.0,286.138655,115.526650,133.0,222.0,268.5,317.0,804.0


Посчитаем среднее и другие характеристики для усеченных данных.

In [14]:
pivot_1 = cut_ds_data.groupby('city')['salary'].describe()
pivot_1

,count,mean,std,min,25%,50%,75%,max
city,,,,,,,,
Moscow,472.0,197.790254,80.950697,71.0,143.75,185.0,250.25,400.0
RU,69.0,118.072464,88.087410,30.0,42.00,98.0,187.00,311.0
St.Petersburg,93.0,105.182796,60.608678,32.0,46.00,99.0,155.00,222.0
West,36.0,360.666667,284.832181,27.0,44.75,366.0,632.50,804.0
exCCCP,44.0,107.727273,72.229725,32.0,47.75,80.0,166.50,257.0


In [15]:
pivot_2 = ds_data.groupby('city')['salary'].describe()
pivot_2

,count,mean,std,min,25%,50%,75%,max
city,,,,,,,,
Moscow,790.0,224.915190,155.770448,30.0,90.25,185.0,323.50,553.0
RU,117.0,145.863248,132.360395,10.0,34.00,98.0,267.00,399.0
St.Petersburg,155.0,122.922581,98.173352,10.0,36.00,99.0,195.50,326.0
West,60.0,401.283333,361.723140,10.0,32.00,366.0,783.25,991.0
exCCCP,75.0,135.666667,121.167556,11.0,37.00,69.0,225.50,388.0


Изменение значения расчета среднего и усеченного среднего (в таблице - специальное):

In [16]:
pivot_1.join(pivot_2,rsuffix='_special')[['mean','mean_special']]

,mean,mean_special
city,,
Moscow,197.790254,224.915190
RU,118.072464,145.863248
St.Petersburg,105.182796,122.922581
West,360.666667,401.283333
exCCCP,107.727273,135.666667


Среднее, ожидаемо, сильно сместилось. Так как мы убрали по 20% самых больших и самых маленьких значений. Самые высокие запрплаты в Москве и на Западе.